In [5]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
from typing import Optional, List, Dict, Any
import pandas as pd
import os
import yaml
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(r"C:\Users\DELL\Documents\vscode_simplon\Brief-2-ETL-de-donn-es-footballistiques-Wickets-Sprinters\.env")

def load_env(env_file: str = ".env") -> None:
    """
    Load environment variables from a .env file.
    Works in both script and notebook environments.
    """

    env_path = Path(env_file)

    if not env_path.is_absolute():
        # Script
        if "__file__" in globals():
            project_root = Path(__file__).resolve().parents[1]
        # Notebook
        else:
            project_root = Path.cwd()

        env_path = project_root / env_path

    if not env_path.exists():
        raise FileNotFoundError(f"❌ .env file not found: {env_path}")
    
    # load_dotenv(env_path)

def fct_load_config(config_filename: str = "config.yaml") -> dict:
    """
    Goal
    ----
    Load configuration parameters from a YAML file.

    This function works both in:
    - Python scripts
    - Jupyter notebooks (where __file__ is not defined)

    Parameters
    ----------
    config_filename : str
        Relative or absolute path to the YAML configuration file.

    Returns
    -------
    dict
        Dictionary containing configuration parameters.
    """

    config_path = Path(config_filename)

    # Cas 1 : chemin absolu → on l'utilise directement
    if config_path.is_absolute():
        final_path = config_path

    else:
        # Cas 2 : Script Python (__file__ existe)
        if "__file__" in globals():
            project_root = Path(__file__).resolve().parents[1]

        # Cas 3 : Notebook Jupyter (__file__ n'existe pas)
        else:
            project_root = Path(os.getcwd())

        final_path = project_root / config_path

    if not final_path.exists():
        raise FileNotFoundError(f"❌ Config file not found: {final_path}")

    with open(final_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    return config


def create_postgres_engine(
    host: str,
    database: str,
    user: str,
    password: str,
    port: int = 5432
) -> Optional[Engine]:
    """
    Crée un moteur SQLAlchemy pour PostgreSQL.

    Paramètres
    ----------
    host : str
        Adresse du serveur PostgreSQL
    database : str
        Nom de la base de données
    user : str
        Utilisateur PostgreSQL
    password : str
        Mot de passe
    port : int, optionnel
        Port PostgreSQL (par défaut 5432)

    Retour
    ------
    sqlalchemy.engine.Engine | None
        Moteur SQLAlchemy ou None en cas d'erreur
    """
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}",
            pool_pre_ping=True
        )
        return engine
    except Exception as e:
        print(f"❌ Erreur de création du moteur SQLAlchemy : {e}")
        return None


def execute_select(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> List[Dict[str, Any]]:
    """
    Exécute une requête SELECT et retourne les résultats sous forme de dictionnaires.
    """
    with engine.connect() as conn:
        result = conn.execute(text(query), params or {})
        return [dict(row._mapping) for row in result]
    
def select_to_dataframe(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> pd.DataFrame:
    """
    Exécute une requête SELECT et retourne un DataFrame pandas.
    """
    return pd.read_sql(text(query), engine, params=params)


def execute_query(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> None:
    """
    Exécute une requête INSERT, UPDATE ou DELETE.
    """
    try:
        with engine.begin() as conn:  # commit automatique
            conn.execute(text(query), params or {})
    except Exception as e:
        print(f"❌ Erreur SQL : {e}")


def dataframe_to_table(
    df: pd.DataFrame,
    table_name: str,
    engine: Engine,
    schema: str = "public",
    if_exists: str = "append"
) -> None:
    """
    Insère un DataFrame pandas dans une table PostgreSQL.
    """
    df.to_sql(
        table_name,
        engine,
        schema=schema,
        if_exists=if_exists,
        index=False,
        method="multi"
    )


In [6]:
host=os.getenv("HOST")
database=os.getenv("DATABASE")
user=os.getenv("USER")
password=os.getenv("PASSWORD")


engine = create_postgres_engine(
    host=host,
    database=database,
    user=user,
    password=password
)

In [7]:
# probabilité de victoire d’une équipe

n_rows = """SELECT COUNT(*) AS nombre_lignes
FROM matches;"""

df_matches = select_to_dataframe(
    engine,
    n_rows
)
df_matches

,nombre_lignes
0,7507


probabilité de victoire d’une équipe

In [18]:
# Probabilité de victoire d’une équipe

req_1 = """WITH finals AS (
    SELECT
        edition,
        CASE
            WHEN home_result > away_result THEN home_team
            ELSE away_team
        END AS winner
    FROM matches
    WHERE stage = 'final'
),
total_cups AS (
    SELECT COUNT(DISTINCT edition) AS total
    FROM finals
),
wins_per_team AS (
    SELECT
        winner AS team,
        COUNT(*) AS titles
    FROM finals
    GROUP BY winner
)
SELECT
    w.team,
    w.titles,
    t.total AS total_world_cups,
    ROUND(w.titles::numeric / t.total, 3) AS probability
FROM wins_per_team w
CROSS JOIN total_cups t
ORDER BY probability DESC;"""

df_matches = select_to_dataframe(
    engine,
    req_1
)


import plotly.express as px

# On suppose que df_matches contient :
# team | titles | total_world_cups | probability

fig = px.bar(
    df_matches.head(9),
    x="team",
    y="probability",
    text="probability",
    title="Probabilité historique de victoire à la Coupe du Monde",
    labels={
        "team": "Équipe",
        "probability": "Probabilité de victoire"
    }
)

# Mise en forme
fig.update_traces(
    texttemplate="%{text:.1%}",
    textposition="outside"
)

fig.update_layout(
    yaxis_tickformat=".0%",
    xaxis_tickangle=-45,
    template="plotly_white",
    bargap=0.2
)

fig.show()

In [16]:
import plotly.graph_objects as go

fig = go.Figure()

# Barres : nombre de titres
fig.add_trace(
    go.Bar(
        x=df_matches["team"],
        y=df_matches["titles"],
        name="Nombre de titres",
        text=df_matches["titles"],
        textposition="auto"
    )
)

# Ligne : probabilité
fig.add_trace(
    go.Scatter(
        x=df_matches["team"],
        y=df_matches["probability"],
        name="Probabilité de victoire",
        yaxis="y2",
        mode="lines+markers",
        text=(df_matches["probability"] * 100).round(1).astype(str) + "%",
        hovertemplate="%{text}"
    )
)

# Layout
fig.update_layout(
    title="Historique des victoires à la Coupe du Monde<br><sup>Titres et probabilité par équipe</sup>",
    xaxis=dict(
        title="Équipe",
        tickangle=-45
    ),
    yaxis=dict(
        title="Nombre de titres"
    ),
    yaxis2=dict(
        title="Probabilité de victoire",
        overlaying="y",
        side="right",
        tickformat=".0%"
    ),
    template="plotly_white",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

In [22]:
# pays ayant marqué le plus en finale
req_2 = """WITH finals AS (
    SELECT
        edition,
        home_team,
        away_team,
        home_result,
        away_result
    FROM matches
    WHERE stage = 'final'
),
goals_by_team AS (
    SELECT
        home_team AS team,
        home_result AS goals
    FROM finals

    UNION ALL

    SELECT
        away_team AS team,
        away_result AS goals
    FROM finals
)
SELECT
    team,
    SUM(goals) AS total_goals_in_finals
FROM goals_by_team
GROUP BY team
ORDER BY total_goals_in_finals DESC
LIMIT 1;
"""

pays_marqué = select_to_dataframe(
    engine,
    req_2
)
pays_marqué

,team,total_goals_in_finals
0,Brazil,28.0


In [23]:
# pays ayant marqué le plus en finale avec classement
req_3 = """SELECT
    team,
    SUM(goals) AS total_goals_in_finals
FROM (
    SELECT home_team AS team, home_result AS goals
    FROM matches
    WHERE stage = 'final'

    UNION ALL

    SELECT away_team AS team, away_result AS goals
    FROM matches
    WHERE stage = 'final'
) t
GROUP BY team
ORDER BY total_goals_in_finals DESC;

"""

pays_marque_sorted = select_to_dataframe(
    engine,
    req_3
)
pays_marque_sorted

,team,total_goals_in_finals
0,Brazil,28.0
1,FRG,11.0
2,Uruguay,11.0
3,Argentina,11.0
4,Italy,11.0
5,France,11.0
6,Sweden,8.0
7,Spain,5.0
8,Hungary,4.0
9,England,4.0


In [19]:
# Probabilité pour UNE équipe donnée
req_2 = """WITH finals AS (
    SELECT
        edition,
        CASE
            WHEN home_result > away_result THEN home_team
            ELSE away_team
        END AS winner
    FROM matches
    WHERE stage = 'final'
)
SELECT
    'France' AS team,
    COUNT(*) AS titles,
    COUNT(*)::numeric / (SELECT COUNT(DISTINCT edition) FROM finals) AS probability
FROM finals
WHERE winner = 'France';
"""

df_matches = select_to_dataframe(
    engine,
    req_1
)
df_matches

,team,titles,total_world_cups,probability
0,Brazil,6,22,0.273
1,France,4,22,0.182
2,Italy,4,22,0.182
3,FRG,3,22,0.136
4,Uruguay,3,22,0.136
5,Spain,2,22,0.091
6,Argentina,2,22,0.091
7,Germany,2,22,0.091
8,Sweden,1,22,0.045
9,WINNER Y,1,22,0.045
